In [ ]:
import os
import pandas as pd

In [ ]:
DATA_DIR = "/resources/data/DecadaVotadaCSV"

In [ ]:
csv_files = [f for f in os.listdir(DATA_DIR) if f.endswith(".csv")]
sorted(csv_files)

## Diputados

### Asuntos

In [ ]:
asuntos_diputados = pd.read_csv(
    os.path.join(DATA_DIR, "asuntos-diputados.csv"), sep=";"
)

In [ ]:
asuntos_diputados.sample(5)

In [ ]:
asuntos_diputados.info()

In [ ]:
# Eliminamos las variables vacías
asuntos_diputados = asuntos_diputados.dropna(axis=1, how="all")
asuntos_diputados.info()

In [ ]:
# Verificamos que asuntoId es único
assert asuntos_diputados["asuntoId"].nunique() == len(asuntos_diputados)

In [ ]:
categorical_columns = asuntos_diputados.select_dtypes(
    include="object"
).columns.tolist()
categorical_columns

In [ ]:
def explore_variable(df: pd.DataFrame, column: str, top_n: int = 20):
    print(f"Explorando la variable '{column}':")
    print(f"- Cantidad de valores únicos: {df[column].nunique()}")
    print(f"- Valores más frecuentes:")
    display(df[column].value_counts().head(top_n))

In [ ]:
for col in categorical_columns:
    explore_variable(asuntos_diputados, col)

In [ ]:
asuntos_diputados.loc[
    (asuntos_diputados["sesion"].str.startswith("142"))
    & (asuntos_diputados["titulo"].str.contains("2643"))
]

### Bloques

In [ ]:
bloques_diputados = pd.read_csv(
    os.path.join(DATA_DIR, "bloques-diputados.csv"), sep=";"
)

In [ ]:
bloques_diputados.sample(5)

In [ ]:
bloques_diputados.info()

In [ ]:
# Verificamos que bloqueId es único
assert bloques_diputados["bloqueId"].nunique() == len(bloques_diputados)

In [ ]:
# Exploramos la cantidad de bloques únicos
bloques_diputados["bloque"].nunique()

In [ ]:
# Identificamos los bloques repetidos
bloques_diputados.loc[
    bloques_diputados["bloque"].duplicated(keep=False)
].sort_values(["bloque", "bloqueId"])

In [ ]:
# Exploramos valores nulos
bloques_diputados.loc[bloques_diputados.isna().any(axis=1)]

### Diputados

In [ ]:
diputados = pd.read_csv(
    os.path.join(DATA_DIR, "diputados-diputados.csv"), sep=";"
)

In [ ]:
diputados.sample(5)

In [ ]:
diputados.info()

In [ ]:
# Verificamos que diputadoId es único
assert diputados["diputadoId"].nunique() == len(diputados)

In [ ]:
# Exploramos la cantidad de diputados únicos
diputados["nombre"].nunique()

In [ ]:
# Buscamos nombres repetidos
diputados.loc[diputados["nombre"].duplicated(keep=False)].sort_values("nombre")

In [ ]:
diputados.loc[diputados["nombre"].duplicated(keep=False)].sort_values(
    "nombre"
).shape[0]

In [ ]:
diputados.loc[
    (diputados["nombre"].duplicated(keep=False))
    & (diputados["imagen"].duplicated(keep=False))
].sort_values(["nombre", "diputadoId"])

In [ ]:
duplicados = set(
    diputados.loc[
        (diputados["nombre"].duplicated(keep=False))
        & (diputados["imagen"].duplicated(keep=False)),
        "diputadoId",
    ]
)

In [ ]:
len(duplicados)

In [ ]:
# Verificamos que los nombres repetidos se corresponden con imágenes iguales
duplicados_nombre_imagen = set(
    diputados.loc[
        (diputados["nombre"].duplicated(keep=False))
        & (diputados["imagen"].duplicated(keep=False)),
        "diputadoId",
    ]
)

assert not duplicados_nombre_imagen.symmetric_difference(duplicados)

In [ ]:
# Exploramos si existen otros casos que comiencen con "#"
diputados.loc[diputados["nombre"].str.startswith("#")]

In [ ]:
# Exploramos los distintos distritos
diputados["distrito"].nunique()

In [ ]:
diputados["distrito"].value_counts()

### Votaciones

In [ ]:
votaciones_diputados = pd.read_csv(
    os.path.join(DATA_DIR, "votaciones-diputados.csv"), sep=";"
)

In [ ]:
votaciones_diputados.sample(5)

In [ ]:
votaciones_diputados.info()

In [ ]:
votaciones_diputados["voto"].value_counts().sort_index()

In [ ]:
voto_4_diputados = votaciones_diputados.loc[
    votaciones_diputados["voto"] == 4, "diputadoId"
].unique()

In [ ]:
diputados.loc[diputados["diputadoId"].isin(voto_4_diputados)].sort_values(
    "diputadoId"
)

In [ ]:
votaciones_diputados.loc[votaciones_diputados["bloqueId"] == 479]

In [ ]:
votaciones_diputados.loc[votaciones_diputados["diputadoId"] == 233]

In [ ]:
votaciones_diputados.loc[votaciones_diputados["diputadoId"] == 2035]

In [ ]:
import os
import sqlite3

In [ ]:
# Definir la ruta de la base de datos local
db_path = "/resources/db"

if not os.path.exists(db_path):
    os.makedirs(db_path)

# Crear la base de datos y una conexión de prueba
conn = sqlite3.connect(os.path.join(db_path, "data.db"))
print(f"Base de datos SQLite creada en: {os.path.join(db_path, 'data.db')}")
conn.close()

In [ ]:
import pandas as pd
import sqlite3
import os

# Definir rutas
DATA_DIR = "/resources/data/DecadaVotadaCSV"
DB_PATH = "/resources/db/data.db"

# Mapear archivos CSV a nombres de tabla
csv_to_table = {
    "asuntos-diputados.csv": "asuntos_diputados",
    "bloques-diputados.csv": "bloques_diputados",
    "diputados-diputados.csv": "legisladores_diputados",
    "votaciones-diputados.csv": "votaciones_diputados",
}

# Conectar a la base de datos
conn = sqlite3.connect(DB_PATH)

for csv_file, table_name in csv_to_table.items():
    df = pd.read_csv(os.path.join(DATA_DIR, csv_file), sep=";")
    df.to_sql(table_name, conn, if_exists="replace", index=False)
    print(f"Tabla '{table_name}' creada con {len(df)} filas.")

conn.close()

In [ ]:
voto_map = {
    0: "AFIRMATIVO",
    1: "NEGATIVO",
    2: "ABSTENCION",
    3: "AUSENTE",
    4: "PRESIDENTE",
}

votaciones_diputados["voto"] = votaciones_diputados["voto"].map(voto_map)
votaciones_diputados["voto"].head()

In [ ]:
import sqlite3

DB_PATH = "/resources/db/data.db"

with sqlite3.connect(DB_PATH) as conn:
    votaciones_diputados.to_sql(
        "votaciones_diputados", conn, if_exists="replace", index=False
    )
    print(
        "Tabla 'votaciones_diputados' actualizada con los valores de voto mapeados."
    )